In [3]:
import cv2 as cv
import sys
import time
import math
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [4]:
class CDI_lib():
    def __init__(self,istream,Threshold=False,mythold=[0,255]):
        super(CDI_lib, self).__init__()
        self.istream=istream
        self.Threshold=Threshold
        self.mythold=mythold
    def CDI(self,start_frame_num,image_type='GRAY',methods='GRAY',scale=False,threshold_type=cv.THRESH_BINARY):
        self.istream.set(cv.CAP_PROP_POS_FRAMES, start_frame_num)
        _, pre_frame = self.istream.read()
        _, frame = self.istream.read()
        _, next_frame = self.istream.read() 
        pre_frame = cv.resize(pre_frame, (224, 224),interpolation=cv.INTER_CUBIC)
        frame = cv.resize(frame, (224, 224),interpolation=cv.INTER_CUBIC)
        next_frame = cv.resize(next_frame, (224, 224),interpolation=cv.INTER_CUBIC)
        if image_type=='GRAY':
            pre_dst=cv.cvtColor(pre_frame, cv.COLOR_BGR2GRAY)
            dst=cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            next_dst=cv.cvtColor(next_frame, cv.COLOR_BGR2GRAY)
        elif image_type=='BGR':
            pre_dst=pre_frame
            dst=frame
            next_dst=next_frame
        elif image_type=='HSV':
            pre_dst=cv.cvtColor(pre_frame, cv.COLOR_BGR2HSV)
            dst=cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            next_dst=cv.cvtColor(next_frame, cv.COLOR_BGR2HSV)

        if methods == 'HARRIS':
            pre_dst = cv.cornerHarris(pre_dst,2,3,0.04)
            dst = cv.cornerHarris(dst,2,3,0.04)
            next_dst = cv.cornerHarris(next_dst,2,3,0.04)
        elif methods == 'LAPLACIAN':
            pre_dst = cv.Laplacian(pre_dst, -1,3)
            dst = cv.Laplacian(dst, -1,3)
            next_dst = cv.Laplacian(next_dst, -1,3)
        elif methods == 'CANNY':
            pre_dst = cv.Canny(pre_dst,50,200)
            dst = cv.Canny(dst,50,200)
            next_dst = cv.Canny(next_dst,50,200)
        elif methods=='OPTICAL':
            _, next_next_frame = self.istream.read()
            next_next_frame = cv.resize(next_next_frame, (224, 224),interpolation=cv.INTER_CUBIC)
            
            fisrt_gray=cv.cvtColor(pre_frame, cv.COLOR_BGR2GRAY)
            second_gray=cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            third_gray=cv.cvtColor(next_frame, cv.COLOR_BGR2GRAY)
            fourth_gray=cv.cvtColor(next_next_frame, cv.COLOR_BGR2GRAY)
            
            pre_flow = cv.calcOpticalFlowFarneback(fisrt_gray,second_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            #pre_dst=np.sqrt(pre_flow[...,0]**2+pre_flow[...,1]**2)
            pre_dst=np.arctan2(pre_flow[...,1],pre_flow[...,0])% (2 * np.pi)
            
            flow = cv.calcOpticalFlowFarneback(second_gray,third_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            #dst=np.sqrt(flow[...,0]**2+flow[...,1]**2)
            dst=np.arctan2(flow[...,1],flow[...,0])% (2 * np.pi)
            
            next_flow=cv.calcOpticalFlowFarneback(third_gray,fourth_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            #next_dst=np.sqrt(next_flow[...,0]**2+next_flow[...,1]**2)
            next_dst=np.arctan2(next_flow[...,1],next_flow[...,0])% (2 * np.pi)
            
        diff=cv.absdiff(pre_dst, dst)
        diff2=cv.absdiff(dst, next_dst)
        
        if scale:
            operator=np.zeros(diff.shape)
            diff=cv.normalize(diff,operator,0,255,cv.NORM_MINMAX)  
            diff2=cv.normalize(diff2,operator,0,255,cv.NORM_MINMAX)  
            
        CDI_IMAGE=cv.bitwise_and(diff,diff2)
        
        if self.Threshold:
            _,diff=cv.threshold(diff, self.mythold[0], self.mythold[1], threshold_type)
            _,diff2=cv.threshold(diff2,self.mythold[0],self.mythold[1],threshold_type)
        
        CDI_IMAGE=cv.bitwise_and(diff,diff2)

            
        return CDI_IMAGE,pre_frame
    @staticmethod 
    def Pos_Morph(Image,OPEN_CLOSE=False,DILATE_ERODE=True,size1=1,size2=1):
        if DILATE_ERODE:
            kernel1 = cv.getStructuringElement(cv.MORPH_RECT,(size1,size1))
            kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(size2,size2))
            Image = cv.erode(Image,kernel1)
            Image =cv.dilate(Image,kernel2)
        elif OPEN_CLOSE:
            kernel1 = cv.getStructuringElement(cv.MORPH_RECT,(size1,size1))
            kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(size2,size2))
            Image = cv.morphologyEx(Image, cv.MORPH_OPEN, kernel1)
            Image = cv.morphologyEx(Image, cv.MORPH_CLOSE, kernel2)
        return Image
    @staticmethod 
    def CDI_Intensity(Image,Normalization=False):
        if Normalization:
            dst=np.zeros(Image.shape)
            Image=cv.normalize(Image,dst,0,1,cv.NORM_MINMAX)
                
        Intensity=np.sum(Image,axis=(1,0))
        return Intensity
    @staticmethod 
    def CDI_Part_Intensity(Image,Normalization=False,bin_size=4):
        img_shape=(4,112,112) if len(Image.shape)==2 else (4,112,112,3)
        if Normalization:
            dst=np.zeros(Image.shape)
            Image=cv.normalize(Image,dst,0,1,cv.NORM_MINMAX)
            
        k=np.array(np.hsplit(Image,2))
        p=np.array([np.vsplit(k[i],2) for i in range(2)]).reshape(img_shape)
        Intensity=np.sum(p,axis=(1,2))
    
        return Intensity
    
    